### Modify path to database in the cell below and run all cells

In [5]:
db_path = '../data_files/temoa_utopia.sqlite'

In [6]:
import nbformat as nbf
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import IPython
from IPython.display import HTML, display, Markdown, Image
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import graphviz
from GraphVizUtil import *
from GraphVizFormats import *
import warnings
warnings.filterwarnings('ignore')

def create_args_flowd(df_graph):
    nodes, tech, ltech, to_tech, from_tech = set(), set(), set(), set(), set()
    for ind,row in df_graph.iterrows():
        if 'ethos' in row['input_comm']:
            ltech.add(row['tech'])
        else :
            nodes.add(row['input_comm'])
        nodes.add(row['output_comm'])
        tech.add(row['tech'])

        if row['input_comm'] != 'ethos':
            to_tech.add('"%s"' % row['input_comm'] + '\t->\t"%s"' % row['tech']) 
        from_tech.add('"%s"' % row['tech'] + '\t->\t"%s"' % row['output_comm'])
    args = dict(
    enodes = "".join('"%s";\n\t\t' % x for x in nodes),
    tnodes = "".join('"%s";\n\t\t' % x for x in tech),
    iedges = "".join('%s;\n\t\t' % x for x in to_tech),
    oedges = "".join('%s;\n\t\t' % x for x in from_tech),
    snodes = ";".join('"%s"' %x for x in ltech),
    )
    return args

def return_format_colors():
    colors = {}
    colors.update(getColorConfig(False))
    return colors, quick_run_dot_fmt

def return_flowd_table(final_dem):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    iterval=0
    while len(inputs)>0:
        df_append = df[df['output_comm'].isin(inputs)]
        df_sel = pd.concat([df_sel, df_append])
        inputs = df_append['input_comm'].unique()
        iterval+=1
        if iterval>2:
            break
    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph


con = sqlite3.connect(db_path) #change path to database
cur = con.cursor()   
con.text_factory = str 

def controls_rows(w):
    controls = HBox(w.children[:-1], layout = Layout(flex_flow='row wrap', width='max-content'))
    output = w.children[-1]
    display(VBox([controls, output],  layout = Layout(flex_flow='columns wrap', width='max-content', height='max-content')))

    display(HTML("<style>.container { width:90% !important; }</style>"))


### Network diagrams for commodities and technologies
Select a table, followed by a sector to view the available commodities and technologies. Then select a specific technology/commodity to generate the network diagram.

In [7]:
def network_sector(sector, table_name):
    if table_name=='commodities':
        query = "SELECT * FROM commodities WHERE flag='p' AND comm_name IN \
    (SELECT DISTINCT(output_comm) FROM efficiency \
    WHERE tech IN (SELECT DISTINCT(tech) FROM technologies WHERE  sector="
    elif table_name=='technologies':
        query = "SELECT * FROM technologies WHERE  sector="

    if sector=='Industry':
        query += "'industrial'"
    elif sector=='Transport':
        query += "'transport'"
    elif sector=='Residential':
        query += "'residential'"
    elif sector=='Commercial':
        query += "'commercial'"
    elif sector=='Supply':
        query += "'supply'"
    elif sector=='Electric':
        query += "'electric'"
    elif sector=='Electric Misc':
        query += "'electric_misc'"
        
    if (table_name=='commodities'):
        query +="))"

    if (sector=='Final demands') & (table_name=='commodities'):
        query = "SELECT * FROM commodities WHERE flag='d'"
    
    if (sector=='Final demands') & (table_name=='technologies'):
        df_demands = pd.DataFrame(columns = pd.read_sql("SELECT * FROM technologies", con).columns)
    else:
        df_demands = pd.read_sql(query, con)
        
    if table_name=='commodities':
        col_filter = 'comm_desc'
    elif table_name=='technologies':
        col_filter = 'tech_desc'

    df_demands[col_filter] = df_demands[col_filter].str.replace('#', '').str.strip()

    def show_desc(desc):
        if desc!='':
            col = col_filter.replace('_desc','').replace('comm','comm_name')
            final_dem = df_demands.loc[df_demands[col_filter]==desc, col].values[0]
            df_graph = return_flowd_table(final_dem)
            args = create_args_flowd(df_graph)
            colors, quick_run_dot_fmt = return_format_colors()
            args.update(colors)
            o_str = 'rankdir = "LR" ;'
            r_str = 'rankdir = "LR" ; \n\t size="12,12";'
            quick_run_dot_fmt = quick_run_dot_fmt.replace(o_str, r_str)
            dot_graph = quick_run_dot_fmt % args
            graph = graphviz.Source(dot_graph)
            display(Markdown('Network diagram for ' + final_dem))
            display(graph)
            
    layout = widgets.Layout(width='500px', height='150px')
    select_options = df_demands[col_filter].unique()
    if len(select_options)==1:
        select_options = list(select_options) + ['']        

    w1 = widgets.Select(options=np.sort(select_options), description=table_name.replace('ies','y').capitalize() , layout=layout)
    w = widgets.interactive(show_desc, desc = w1)
    controls_rows(w)

#selects particular table
def select_tech_comm():
    def single_table(table_val, sector):
        if table_val=='Technologies':
            network_sector(sector, 'technologies')
        elif table_val=='Commodities':
            network_sector(sector, 'commodities')
            
    layout = widgets.Layout(width='200px', height = '50px')
    w1 = widgets.Select(options=['Technologies', 'Commodities'], \
                        description='Table', layout=layout)
    layout = widgets.Layout(width='200px', height = '150px')
    w2 = widgets.Select(options=['Final demands','Supply','Transport','Residential','Commercial','Industry', 'Electric', 'Electric Misc',], \
                        description='Sector', layout=layout)
    w = widgets.interactive(single_table, table_val = w1, sector = w2)
    controls_rows(w)

#main function
select_tech_comm()

### Technology/commodity look-up tool
Use the tool below to provide a description for any technology or commodity within the database. Type the commodity or technology name in the box below to view its description.

In [8]:
w = widgets.Text(value='ELC')
display(w)
def f(w):
    df = pd.read_sql("SELECT * FROM commodities WHERE comm_name='" + w + "'", con)
    if len(df)==0:
        df = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)

    if len(df)>0:
        try:
            display(Markdown(df['comm_desc'].values[0].replace('#', '').strip()))
        except:
            display(Markdown(df['tech_desc'].values[0].replace('#', '').strip()))
    else:
        print('')


out = widgets.interactive_output(f, {'w': w})
display(out)


Text(value='ELC')

Output()